In [1]:
import requests
from bs4 import BeautifulSoup
from itertools import zip_longest
import csv
import re
import pandas as pd

In [2]:
request = requests.get('https://wuzzuf.net/search/jobs/?q=data+analyst&a=navbl')
request

<Response [200]>

In [3]:
source = request.content
source[:200]

b'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, s'

In [4]:
soup = BeautifulSoup(source, 'html.parser')

#           

In [5]:
job_titles = soup.find_all('h2', class_="css-m604qf")
job_titles[-2].text

'Business Analyst'

In [6]:
company_location = soup.find_all('span', class_='css-5wys0k')
company_location[-2].text

'Heliopolis, Cairo, Egypt '

In [7]:
company_names = soup.find_all('a', class_='css-17s97q8')
company_names[-2].text

'MTS -'

In [8]:
job_skills = soup.find_all('div', class_='css-y4udm8')
job_skills[-2].text

'Full TimeExperienced · 2 - 4 Yrs of Exp · IT/Software Development · Analyst/Research · Engineering - Telecom/Technology · Analysis · business · Business Analysis · Communication skills · Computer Science · Data · English'

In [9]:
job_types = soup.find_all('span', class_='css-1ve4b75 eoyjyou0')
job_types[-2].text

'Full Time'

In [10]:
time = (soup.find_all('div', class_="css-4c4ojb")) + (soup.find_all('div', class_="css-do6t5g"))
time[-2].text

'12 days ago'

In [11]:
[i.text for i in time]

['2 days ago',
 '2 days ago',
 '1 day ago',
 '5 days ago',
 '2 days ago',
 '13 days ago',
 '18 days ago',
 '29 days ago',
 '26 days ago',
 '27 days ago',
 '30 days ago',
 '1 month ago',
 '13 days ago',
 '12 days ago',
 '1 month ago']

#         

In [17]:
job_title = []
job_skill = []
company_locat = []
company_name = []
job_type = []
posted_time = []

In [18]:
for i in range(len(job_titles)):
    job_title.append(job_titles[i].text)
    job_skill.append(job_skills[i].text)
    company_locat.append(company_location[i].text)
    company_name.append(company_names[i].text)
    job_type.append(job_types[i].text)
    posted_time.append(time[i].text)

#        

> We can observe that there are some problems in some columns like job_title, company_name and job_skill, so we need to make a little cleaning

#         

### Cleaning  (job_title),  (job_skill), (company_name)

In [19]:
job_title

['.css-o171kl{-webkit-text-decoration:none;text-decoration:none;color:inherit;}Data Analyst',
 'Data Analyst',
 'Data Analyst Supervisor',
 'Senior Digital Data Analyst',
 'Data Analyst',
 'Data Analyst',
 'Senior Data & Web Analyst',
 'Data analyst',
 'Data Analyst',
 'Financial Data Analyst - (French/Russian/German/Italy) Speakers',
 'English Financial Data Analyst',
 'Data Analyst',
 'Business Analyst ( Project Base Onsite )',
 'Business Analyst',
 'Data Analyst']

In [28]:
job_title = list(map(lambda x :(re.sub('\.css-\w+{.*}', '', x)), job_title))
job_title

['Data Analyst',
 'Data Analyst',
 'Data Analyst Supervisor',
 'Senior Digital Data Analyst',
 'Data Analyst',
 'Data Analyst',
 'Senior Data & Web Analyst',
 'Data analyst',
 'Data Analyst',
 'Financial Data Analyst - (French/Russian/German/Italy) Speakers',
 'English Financial Data Analyst',
 'Data Analyst',
 'Business Analyst ( Project Base Onsite )',
 'Business Analyst',
 'Data Analyst']

#         

In [30]:
job_skill[0]

'.css-1lh32fc{display:-webkit-box;display:-webkit-flex;display:-ms-flexbox;display:flex;margin-bottom:8px;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-box-flex-wrap:wrap;-webkit-flex-wrap:wrap;-ms-flex-wrap:wrap;flex-wrap:wrap;}.css-n2jc4m{display:-webkit-inline-box;display:-webkit-inline-flex;display:-ms-inline-flexbox;display:inline-flex;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;-webkit-text-decoration:none;text-decoration:none;color:inherit;margin-bottom:4px;}.css-adtuo7{cursor:pointer;padding:0 4px;border-radius:4px;}.css-1ve4b75{font-size:12px;font-weight:600;display:-webkit-inline-box;display:-webkit-inline-flex;display:-ms-inline-flexbox;display:inline-flex;-webkit-align-items:center;-webkit-box-align:center;-ms-flex-align:center;align-items:center;min-height:20px;margin-right:4px;border-radius:2px;max-width:196px;white-space:nowrap;overflow:hidden;cursor:default;text-overf

In [33]:
job_skill= list(map(lambda x: re.sub('.css-\w+{.+}', '', x), job_skill))
job_skill[0]

' · Information Technology (IT) · BI · Analysis · business · analytical · SQL · Matplotlib'

#      

In [112]:
company_name

['Nahr Development -',
 'Orient -',
 'SURE International Technology -',
 'Jazeera Paints -',
 'Grand Plast  -',
 'Alarabia Group -',
 'Finstek -',
 'Confidential -',
 'Rashideen Egypt for Trade  -',
 'World Vest Base (WVB ) -',
 'World Vest Base (WVB ) -',
 'MTS Logistics -',
 'Qsource -',
 'MTS -',
 'Confidential -']

In [113]:
company_name = list(map(lambda x: x.replace('-', '').strip() , company_name))
company_name

['Nahr Development',
 'Orient',
 'SURE International Technology',
 'Jazeera Paints',
 'Grand Plast',
 'Alarabia Group',
 'Finstek',
 'Confidential',
 'Rashideen Egypt for Trade',
 'World Vest Base (WVB )',
 'World Vest Base (WVB )',
 'MTS Logistics',
 'Qsource',
 'MTS',
 'Confidential']

#        

In [114]:
file_list = [job_title, posted_time, company_name, company_locat, job_type, job_skill]
exported_list = zip_longest(*file_list)
with open('test.csv', 'w', encoding='utf-8') as file:
    writer_obj = csv.writer(file)
    writer_obj.writerow(['job_title', 'posted_time', 'company_name','company_location', 'job_type', 'job_skills'])
    writer_obj.writerows(exported_list)

In [116]:
data = pd.read_csv('test.csv')
data

,job_title,posted_time,company_name,company_location,job_type,job_skills
0,Data Analyst,2 days ago,Nahr Development,"Bolkly, Alexandria, Egypt",Full Time,· Information Technology (IT) · BI · Analysis...
1,Data Analyst,2 days ago,Orient,"Maadi, Cairo, Egypt",Work From Home,Full TimeEntry Level · 1+ Yrs of Exp · Analyst...
2,Data Analyst Supervisor,1 day ago,SURE International Technology,"Cairo, Egypt",Full Time,Full TimeManager · 5+ Yrs of Exp · IT/Software...
3,Senior Digital Data Analyst,5 days ago,Jazeera Paints,"10th of Ramadan City, Cairo, Egypt",Full Time,Full TimeExperienced · 3 - 6 Yrs of Exp · IT/S...
4,Data Analyst,2 days ago,Grand Plast,"Salam City, Cairo, Egypt",Full Time,Full TimeExperienced · 7 - 12 Yrs of Exp · Ana...
5,Data Analyst,13 days ago,Alarabia Group,"10th of Ramadan City, Cairo, Egypt",Full Time,Full TimeExperienced · 2 - 5 Yrs of Exp · IT/S...
6,Senior Data & Web Analyst,18 days ago,Finstek,"New Cairo, Cairo, Egypt",Full Time,Full TimeExperienced · 3+ Yrs of Exp · IT/Soft...
7,Data analyst,29 days ago,Confidential,"Qalyub, Qalubia, Egypt",Full Time,Full TimeExperienced · 2 - 4 Yrs of Exp · male...
8,Data Analyst,26 days ago,Rashideen Egypt for Trade,"Basatin, Cairo, Egypt",Full Time,Full TimeExperienced · 2 - 5 Yrs of Exp · Busi...
9,Financial Data Analyst - (French/Russian/Germa...,27 days ago,World Vest Base (WVB ),"Heliopolis, Cairo, Egypt",Full Time,Full TimeEntry Level · 0 - 2 Yrs of Exp · Acco...


In [117]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   job_title         15 non-null     object
 1   posted_time       15 non-null     object
 2   company_name      15 non-null     object
 3   company_location  15 non-null     object
 4   job_type          15 non-null     object
 5   job_skills        15 non-null     object
dtypes: object(6)
memory usage: 424.0+ bytes


In [125]:
data['job_skills'][12]

'Freelance / ProjectEntry Level · 1 - 2 Yrs of Exp · Analyst/Research · Agile · Analysis · Business Analysis · business · Computer Science · Development · Engineering · Data · Information Technology (IT)'

#      